<a href="https://colab.research.google.com/github/jvishnuvardhan/Code_snippets/blob/master/via_to_coco_conversion_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. VIA to COCO format conversion:

In [ ]:
from pprint import pprint

In [ ]:
import os
import skimage 
import math
from itertools import chain
import numpy as np
import json

# dataset_dir
# vgg_path = './annotations_via_sample2.json'
vgg_path_coco = '/content/train_coco.json'
vgg_path = '/content/train_via_json.json'
class_keyword = "names" #"Material Type"
# subclass_keyword = "" #"Product Type"

with open(vgg_path) as f:
    vgg = json.load(f)

In [ ]:
# to access height and width from coco json file
with open(vgg_path_coco) as f:
    vgg_coco = json.load(f)

images_coco = vgg_coco['images']

height_width =[]
for image in images_coco:
  # print(image)
  height_width.append([image['height'], image['width']])

print(height_width)

In [ ]:
list(vgg.keys())

['359925415_fb5ea66be1_c.jpg137958',
 '404523005_f50e859da7_c.jpg144686',
 '2012120328_fd5c574885_c.jpg162831',
 '2928182339_57d9422d1d_c.jpg63838',
 '2928182583_1ac4d4e4b4_c.jpg96005',
 '6476255913_a1dccd4ab2_z.jpg82431',
 '6592413485_b54b0265bf_c.jpg93180',
 '7979493761_a91a5496cd_c.jpg163279',
 '10647071755_c864af9a57_c.jpg74111',
 '786678226_2971274d0c_c.jpg148474',
 '1302119263_9da3dc23eb_c.jpg163347',
 '1302871965_feca1e69c3_c.jpg184689',
 '1302906767_b2e92de824_c.jpg107747',
 '2245393517_6804a8184b_c.jpg105624',
 '2958135246_bbe9591e55_c.jpg144026',
 '14359024134_8d568f293f_c.jpg158380',
 '26276953382_579c1d263e_c.jpg117850',
 '47830670502_c612edd7bb_c.jpg179032']

In [ ]:
# vgg['images'][:2]

In [ ]:
# print(images)

In [ ]:
print(vgg.values())

dict_values([{'filename': '359925415_fb5ea66be1_c.jpg', 'size': 137958, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [312, 618, 638, 675, 704, 715, 750, 765, 762, 718, 667, 512, 391, 310, 243, 150, 80, 55, 28, 24, 45, 118, 185, 202, 227, 276, 293], 'all_points_y': [405, 417, 442, 449, 419, 409, 394, 361, 301, 228, 207, 184, 182, 201, 238, 242, 253, 267, 298, 332, 367, 394, 401, 432, 450, 440, 427]}, 'region_attributes': {'names': 'Car'}}], 'file_attributes': {}}, {'filename': '404523005_f50e859da7_c.jpg', 'size': 144686, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [135, 290, 440, 476, 499, 529, 539, 675, 697, 711, 720, 714, 667, 657, 556, 428, 307, 211, 141, 123, 109, 117], 'all_points_y': [422, 459, 450, 479, 486, 457, 422, 360, 373, 365, 309, 265, 243, 227, 184, 179, 201, 249, 267, 325, 349, 383]}, 'region_attributes': {'names': 'Car'}}], 'file_attributes': {}}, {'filename': '2012120328_fd5c574885_c.jpg', 'size': 162831, 'regions': [{'

In [ ]:
# pprint(vgg)

In [ ]:
images_ids_dict = {}
images_info = []

for i,v in enumerate(vgg.values()):
  # print(v)
  images_ids_dict[v["filename"]] = i
  height, width = height_width[i] #[1080, 1920]  
  images_info.append({"file_name": v["filename"], "id": i, "width": width, "height": height})

# when subclass exists    
# classes = {class_keyword} | {r["region_attributes"][class_keyword]+"_"+r["region_attributes"][subclass_keyword]
#                              for v in vgg.values() 
#                              if v["regions"]!=[]
#                              for r in v["regions"]
#                          if (class_keyword in r["region_attributes"].keys() and subclass_keyword in r["region_attributes"].keys())}

# when subclass doesn't exist
classes = {r["region_attributes"][class_keyword]
                             for v in vgg.values() 
                             if v["regions"]!=[]
                             for r in v["regions"]
                         if (class_keyword in r["region_attributes"].keys())}
                         
category_ids_dict = {c: i for i, c in enumerate(classes, 1)}
categories = [{"supercategory": class_keyword, "id": v, "name": k} for k, v in category_ids_dict.items()]

In [ ]:
print(classes)

{'Car', 'Bus'}


In [ ]:
category_ids_dict

{'Bus': 2, 'Car': 1}

In [ ]:
annotations = []
suffix_zeros = math.ceil(math.log10(len(vgg)))

def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

for i, v in enumerate(vgg.values()):
    for j, r in enumerate(v["regions"]):
        if class_keyword in r["region_attributes"]: #and subclass_keyword in r["region_attributes"]:
            x, y = r["shape_attributes"]["all_points_x"], r["shape_attributes"]["all_points_y"]
            annotations.append({
                "segmentation": [list(chain.from_iterable(zip(x, y)))],
                "area": PolyArea(x, y),
                "bbox": [min(x), min(y), max(x)-min(x), max(y)-min(y)],
                "image_id": images_ids_dict[v["filename"]],
                "category_id": category_ids_dict[r["region_attributes"][class_keyword]], #"category_id": category_ids_dict[r["region_attributes"][class_keyword]+"_"+r["region_attributes"][subclass_keyword]],
                "id": int(f"{i:0>{suffix_zeros}}{j:0>{suffix_zeros}}"),
                "iscrowd": 0
                })

In [ ]:
coco = {
    "images": images_info,
    "categories": categories,
    "annotations": annotations
    }

outfile = vgg_path.replace("via", "coco")

with open(outfile, "w") as f:
    json.dump(coco, f)

**COCO to VIA**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [ ]:
 import json

# coco = './coco_format_conversion/annotations_coco_sample1.json'
coco = '/content/train_coco_json.json' # via annotations
# via = './coco_format_conversion/annotations_via_sample1.json'

with open(coco) as f:
  vgg1 = json.load(f)

# with open(via) as f:
#     vgg2 = json.load(f)

In [ ]:
vgg1.keys()

dict_keys(['images', 'categories', 'annotations'])

In [ ]:
vgg1['images']

[{'file_name': '359925415_fb5ea66be1_c.jpg',
  'height': 533,
  'id': 0,
  'width': 799},
 {'file_name': '404523005_f50e859da7_c.jpg',
  'height': 600,
  'id': 1,
  'width': 800},
 {'file_name': '2012120328_fd5c574885_c.jpg',
  'height': 533,
  'id': 2,
  'width': 800},
 {'file_name': '2928182339_57d9422d1d_c.jpg',
  'height': 600,
  'id': 3,
  'width': 800},
 {'file_name': '2928182583_1ac4d4e4b4_c.jpg',
  'height': 600,
  'id': 4,
  'width': 800},
 {'file_name': '6476255913_a1dccd4ab2_z.jpg',
  'height': 480,
  'id': 5,
  'width': 640},
 {'file_name': '6592413485_b54b0265bf_c.jpg',
  'height': 513,
  'id': 6,
  'width': 800},
 {'file_name': '7979493761_a91a5496cd_c.jpg',
  'height': 600,
  'id': 7,
  'width': 800},
 {'file_name': '10647071755_c864af9a57_c.jpg',
  'height': 450,
  'id': 8,
  'width': 800},
 {'file_name': '786678226_2971274d0c_c.jpg',
  'height': 600,
  'id': 9,
  'width': 800},
 {'file_name': '1302119263_9da3dc23eb_c.jpg',
  'height': 536,
  'id': 10,
  'width': 800},


In [ ]:
# vgg2.keys()

dict_keys(['20210826-105458.png', '20210826-105503.png', '20210826-105512.png', '20210826-105514.png', '20210826-105521.png', '20210826-105529.png', '20210826-105532.png', '20210826-105534.png', '20210826-105601.png', '20210826-105603.png', '20210826-105635.png', '20210826-105639.png', '20210826-105646.png', '20210826-105651.png', '20210826-105656.png', '20210826-105700.png', '20210826-105704.png', '20210826-105706.png', '20210826-105712.png', '20210826-105718.png', '20210826-105724.png', '20210826-105727.png', '20210826-105729.png', '20210826-105743.png', '20210826-105801.png', '20210826-105805.png', '20210826-105826.png', '20210826-105830.png', '20210826-105842.png', '20210826-105844.png', '20210826-105850.png', '20210826-105853.png', '20210826-105856.png', '20210826-105901.png', '20210826-105906.png', '20210826-105912.png', '20210826-105916.png', '20210826-105921.png', '20210826-105924.png', '20210826-105926.png', '20210826-105928.png', '20210826-105931.png', '20210826-105935.png', 

In [ ]:
# vgg2['20210826-105458.png']

{'file_attributes': {},
 'filename': '20210826-105458.png',
 'regions': [{'region_attributes': {'Material Type': 'PET',
    'Product Type': 'alcohol Bottles'},
   'shape_attributes': {'all_points_x': [1185,
     1161,
     1148,
     1165,
     1246,
     1271,
     1294,
     1319,
     1290,
     1292,
     1222,
     1200],
    'all_points_y': [140,
     170,
     199,
     216,
     263,
     256,
     269,
     241,
     227,
     199,
     158,
     142],
    'name': 'polygon'}},
  {'region_attributes': {'Material Type': 'HDPE-Rigid',
    'Product Type': 'Food Packing '},
   'shape_attributes': {'all_points_x': [1319,
     1281,
     1253,
     1255,
     1249,
     1306,
     1332,
     1364,
     1417,
     1441,
     1404,
     1358,
     1338],
    'all_points_y': [297,
     326,
     365,
     387,
     424,
     477,
     485,
     464,
     427,
     372,
     315,
     293,
     291],
    'name': 'polygon'}},
  {'region_attributes': {'Material Type': 'Others',
    'Produc

In [ ]:
# !ls ./coco_format_conversion/sample_images1/ | wc -l

48


In [ ]:
# sample2 - 271
# sample1 - 48

**Get the material type only from JSON file**

In [ ]:
# file = './coco_format_conversion/annotations_coco_sample1.json'
# with open(file) as f:
#   coco = json.load(f)

In [ ]:
#  import json

# file = './coco_format_conversion/annotations_via_sample1.json'

# with open(file) as f:
#   vgg = json.load(f)

In [ ]:
# import tqdm
# l = []
# for i in tqdm.tqdm(vgg.keys()):
#   # print(i)
#   for j in range(len(vgg[i]['regions'])):
#     # print(j)
#     if 'Material Type' in vgg[i]['regions'][j]['region_attributes'].keys() and not bool(vgg[i]['regions'][j]['region_attributes']) == False:
#       l.append(vgg[i]['regions'][j]['region_attributes']['Material Type'])


100%|██████████| 271/271 [00:00<00:00, 35797.95it/s]


In [ ]:
# import numpy as np
# np.unique(np.unique(l)),len(np.unique(np.unique(l)))

(array(['HDPE-Rigid', 'Others', 'PET', 'PP-Flexible'], dtype='<U11'), 4)

In [ ]:
# import numpy as np
# np.unique(np.unique(l)),len(np.unique(np.unique(l)))

(array(['HDPE-Flexible', 'HIPC', 'LDPE-Flexible', 'MLP', 'Others', 'PET',
        'PP-Flexible', 'Unknown'], dtype='<U13'), 8)

In [ ]:
# vgg.keys()

dict_keys(['20210826-105458.png', '20210826-105503.png', '20210826-105512.png', '20210826-105514.png', '20210826-105521.png', '20210826-105529.png', '20210826-105532.png', '20210826-105534.png', '20210826-105601.png', '20210826-105603.png', '20210826-105635.png', '20210826-105639.png', '20210826-105646.png', '20210826-105651.png', '20210826-105656.png', '20210826-105700.png', '20210826-105704.png', '20210826-105706.png', '20210826-105712.png', '20210826-105718.png', '20210826-105724.png', '20210826-105727.png', '20210826-105729.png', '20210826-105743.png', '20210826-105801.png', '20210826-105805.png', '20210826-105826.png', '20210826-105830.png', '20210826-105842.png', '20210826-105844.png', '20210826-105850.png', '20210826-105853.png', '20210826-105856.png', '20210826-105901.png', '20210826-105906.png', '20210826-105912.png', '20210826-105916.png', '20210826-105921.png', '20210826-105924.png', '20210826-105926.png', '20210826-105928.png', '20210826-105931.png', '20210826-105935.png', 

In [3]:
# vgg['20210826-105458.png']

In [ ]:
# name = '20210826-105458.png'
# vgg[name].keys()

dict_keys(['filename', 'size', 'regions', 'file_attributes'])

In [ ]:
# gg = {}
# gg['20210826-105458.png'] = {}
# gg['20210826-105458.png']['file_attributes'] = {}
# gg['20210826-105458.png']['filename'] = '20210826-105458.png'
# regions_list = []
# di = {}
# di['region_attributes'] = {'Material Type':}
# gg['20210826-105458.png']['regions'] = regions_list
# gg['20210826-105458.png']['size'] = -1
# gg

In [1]:
# import tqdm
# l = []
# for i in tqdm.tqdm(vgg.keys()):
#   print(i)
#   for j in vgg[i]['regions']:
#     print(j)
#     if 'Material Type' in j['region_attributes'].keys() and not bool(j['region_attributes']) == False:
#       l.append(j['region_attributes']['Material Type'])
#     else:
#       del j

In [2]:
# import tqdm
# l = []
# for i in tqdm.tqdm(vgg.keys()):
#   print(i)
#   for j in range(len(vgg[i]['regions'])):
#     print(j)
#     # if 'Material Type' in vgg[i]['regions'][j]['region_attributes'].keys() and not bool(vgg[i]['regions'][j]['region_attributes']) == False:
#     l.append(vgg[i]['regions'][j]['region_attributes']['Material Type'])
#     # else:
#     #   del vgg[i]['regions'][j]

In [ ]:
vgg['20210826-105532.png']['regions']

IndexError: ignored